In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,desc,sum,avg,array_contains, when
from pyspark.sql.types import StructType, StructField, StringType

In [49]:
sc = SparkSession.builder \
    .master("spark://ravi:7077") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .getOrCreate()

In [50]:
books = sc.read.csv('hdfs://ravi:9000/Books.csv',header=True)
users = sc.read.csv('hdfs://ravi:9000/Users.csv',header=True)
ratings = sc.read.csv('hdfs://ravi:9000/Ratings.csv',header=True)

In [51]:
books.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [52]:
users.show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|18.0|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|17.0|
|      5|farnborough, hant...|NULL|
|      6|santa monica, cal...|61.0|
|      7| washington, dc, usa|NULL|
|      8|timmins, ontario,...|NULL|
|      9|germantown, tenne...|NULL|
|     10|albacete, wiscons...|26.0|
|     11|melbourne, victor...|14.0|
|     12|fort bragg, calif...|NULL|
|     13|barcelona, barcel...|26.0|
|     14|mediapolis, iowa,...|NULL|
|     15|calgary, alberta,...|NULL|
|     16|albuquerque, new ...|NULL|
|     17|chesapeake, virgi...|NULL|
|     18|rio de janeiro, r...|25.0|
|     19|           weston, ,|14.0|
|     20|langhorne, pennsy...|19.0|
+-------+--------------------+----+
only showing top 20 rows



In [53]:
ratings.show()

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
| 276733|2080674722|          0|
| 276736|3257224281|          8|
| 276737|0600570967|          6|
| 276744|038550120X|          7|
| 276745| 342310538|         10|
| 276746|0425115801|          0|
| 276746|0449006522|          0|
| 276746|0553561618|          0|
| 276746|055356451X|          0|
| 276746|0786013990|          0|
| 276746|0786014512|          0|
| 276747|0060517794|          9|
| 276747|0451192001|          0|
| 276747|0609801279|          0|
| 276747|0671537458|          9|
+-------+----------+-----------+
only showing top 20 rows



In [54]:
books_rows = books.count()
books_columns = len(books.columns)
print("Books DataFrame shape: ({}, {})".format(books_rows, books_columns))

Books DataFrame shape: (271360, 8)


In [55]:
ratings_rows = ratings.count()
ratings_columns = len(ratings.columns)
print("Ratings DataFrame shape: ({}, {})".format(ratings_rows, ratings_columns))

Ratings DataFrame shape: (1149780, 3)


In [56]:
users_rows = users.count()
users_columns = len(users.columns)
print("Users DataFrame shape: ({}, {})".format(users_rows, users_columns))

Users DataFrame shape: (278859, 3)


In [57]:

# Calculate the count of null values for each column in the books DataFrame
null_counts = books.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in books.columns])

# Print the count of null values for each column
null_counts.show()

+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|ISBN|Book-Title|Book-Author|Year-Of-Publication|Publisher|Image-URL-S|Image-URL-M|Image-URL-L|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|   0|         0|          1|                  0|        2|          0|          0|          3|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+



In [58]:
# Calculate the count of null values for each column in the books DataFrame
null_count = users.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in users.columns])

# Print the count of null values for each column
null_count.show()

+-------+--------+------+
|User-ID|Location|   Age|
+-------+--------+------+
|      1|       0|110518|
+-------+--------+------+



In [59]:
# Calculate the count of null values for each column in the books DataFrame
null_con = ratings.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in ratings.columns])

# Print the count of null values for each column
null_con.show()

+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+



In [60]:
from pyspark.sql.functions import count, lit

# Count the number of duplicate rows in the books DataFrame
duplicate_counts = (
    books.select("*")
    .groupBy(*books.columns)
    .agg(count(lit(1)).alias("count"))
    .filter("count > 1")
    .agg(sum("count").alias("total_duplicates"))
    .collect()[0]["total_duplicates"]
)

print(f"Number of duplicate rows: {duplicate_counts}")

Number of duplicate rows: None


In [61]:
# Count the number of duplicate rows in the books DataFrame
duplicate_count= (
    users.select("*")
    .groupBy(*users.columns)
    .agg(count(lit(1)).alias("count"))
    .filter("count > 1")
    .agg(sum("count").alias("total_duplicates"))
    .collect()[0]["total_duplicates"]
)

print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: None


In [62]:
# Count the number of duplicate rows in the books DataFrame
duplicate_con= (
    ratings.select("*")
    .groupBy(*ratings.columns)
    .agg(count(lit(1)).alias("count"))
    .filter("count > 1")
    .agg(sum("count").alias("total_duplicates"))
    .collect()[0]["total_duplicates"]
)

print(f"Number of duplicate rows: {duplicate_con}")

Number of duplicate rows: None


In [63]:
####Popularity Based Recommendation System

In [64]:
ratings_with_name = ratings.join(books, ratings.ISBN == books.ISBN, "inner").drop(books.ISBN)

In [65]:
num_rating_df = ratings_with_name.groupBy('Book-Title').agg(count(col('Book-Rating')).alias('num_ratings'))

# Show the resulting DataFrame
num_rating_df.show()

+--------------------+-----------+
|          Book-Title|num_ratings|
+--------------------+-----------+
|Survival Guide to...|          1|
|              Apt. 3|          1|
|     In the Clearing|          2|
|Too Many Men : A ...|          4|
|Circles of Confus...|          1|
|Emily Post's Wedd...|          2|
|Two of a Kind #15...|          1|
|Speculations: The...|          1|
|The broom of the ...|          2|
|  Fran Ellen's House|          2|
|Queen of Spades a...|          1|
|Cooking in a Beds...|          2|
|Witnesses to War:...|          1|
|The Dream of the ...|          2|
|The Wizard's Holi...|          3|
|Madness and Sexua...|          1|
|    Rapture's Legacy|          2|
|Fallen Hero/the S...|          1|
|Red Earth and Pou...|          1|
|Last Train to Mem...|          5|
+--------------------+-----------+
only showing top 20 rows



In [66]:
avg_rating_df = ratings_with_name.groupBy('Book-Title').agg(avg('Book-Rating').alias('avg_ratings'))

# Show the resulting DataFrame
avg_rating_df.show()

+--------------------+------------------+
|          Book-Title|       avg_ratings|
+--------------------+------------------+
|Survival Guide to...|               0.0|
|              Apt. 3|               0.0|
|     In the Clearing|               5.0|
|Too Many Men : A ...|               0.0|
|Circles of Confus...|               0.0|
|Emily Post's Wedd...|               4.5|
|Two of a Kind #15...|               0.0|
|Speculations: The...|               0.0|
|The broom of the ...|               4.0|
|  Fran Ellen's House|               0.0|
|Queen of Spades a...|              10.0|
|Cooking in a Beds...|               6.5|
|Witnesses to War:...|               0.0|
|The Dream of the ...|               1.5|
|The Wizard's Holi...|3.3333333333333335|
|Madness and Sexua...|               5.0|
|    Rapture's Legacy|               0.0|
|Fallen Hero/the S...|               0.0|
|Red Earth and Pou...|               0.0|
|Last Train to Mem...|               3.6|
+--------------------+------------

In [67]:
# Perform join operation
popular_df = num_rating_df.join(avg_rating_df, 'Book-Title', 'inner')

# Show the resulting DataFrame
popular_df.show()

+--------------------+-----------+------------------+
|          Book-Title|num_ratings|       avg_ratings|
+--------------------+-----------+------------------+
| A Light in the S...|          4|              2.25|
| Ask Lily (Young ...|          1|               8.0|
| Flight of Fancy:...|          2|               4.0|
| Murder of a Slee...|         11|3.1818181818181817|
| Q-Space (Star Tr...|         17|1.9411764705882353|
| Summonings: Book...|          2|               7.0|
| This Place Has N...|          9|               2.0|
|                !Yo!|         11| 5.181818181818182|
|"A Wing and a Pra...|          1|               7.0|
|"Active Drama Pla...|          2|               3.0|
|"Best Places Nort...|          1|               0.0|
|"Best \Thinking M...|          3|2.3333333333333335|
|"Black Sheep One ...|          2|               0.0|
|"Cannery Row (Ste...|          2|               3.5|
|"Cinderella and O...|          5|               1.8|
|"Deslices histÃ³r...|      

In [68]:
# # Filter and sort popular_df
# popular_df = popular_df.filter(col('num_ratings') >= 250) \
#     .sort(col('avg_ratings').desc()) \
#     .limit(50)

In [ ]:
# popular_df = popular_df.join(books, on='Book-Title', how='left') \
#                        .dropDuplicates(['Book-Title']) \
#                        .select('Book-Title', 'Book-Author','Image-URL-L','num_ratings', 'avg_ratings')

In [ ]:
from pyspark.sql.functions import col, desc

# Filter and sort popular_df
popular_df = popular_df.filter(col('num_ratings') >= 250) \
                        .sort(col('avg_ratings').desc()) \
                        .limit(50)

popular_df = popular_df.join(books, on='Book-Title', how='left') \
                        .dropDuplicates(['Book-Title']) \
                        .select('Book-Title', 'books.Book-Author', 'Image-URL-L', 'num_ratings', 'avg_ratings')

# Print the DataFrame sorted in descending order of avg_ratings
popular_df.sort(desc('avg_ratings')).show()

In [85]:
popular_df.show()

+--------------------+------------------+--------------------+-----------+------------------+
|          Book-Title|       Book-Author|         Image-URL-L|num_ratings|       avg_ratings|
+--------------------+------------------+--------------------+-----------+------------------+
|Harry Potter and ...|     J. K. Rowling|http://images.ama...|        428| 5.852803738317757|
|Harry Potter and ...|     J. K. Rowling|http://images.ama...|        387|5.8242894056847545|
|Harry Potter and ...|     J. K. Rowling|http://images.ama...|        278| 5.737410071942446|
|Harry Potter and ...|     J. K. Rowling|http://images.ama...|        347| 5.501440922190202|
|Harry Potter and ...|     J. K. Rowling|http://images.ama...|        556| 5.183453237410072|
|The Hobbit : The ...|    J.R.R. TOLKIEN|http://images.ama...|        281|  5.00711743772242|
|The Fellowship of...|    J.R.R. TOLKIEN|http://images.ama...|        368| 4.948369565217392|
|Harry Potter and ...|     J. K. Rowling|http://images.ama..

In [71]:
##Collaborative Filtering Based Recommender System

In [72]:
# Create a DataFrame with the count of 'Book-Rating' grouped by 'User-ID'
count_df = ratings_with_name.groupBy('User-ID').count().withColumnRenamed('count', 'Book-Rating')

# Filter the DataFrame to keep only rows where 'Book-Rating' is greater than 200
x = count_df.filter(count_df['Book-Rating'] > 200)

# Get the 'User-ID' values from the filtered DataFrame
y = x.select('User-ID').rdd.flatMap(lambda x: x).collect()

In [73]:
filtered_rating = ratings_with_name.filter(col('User-ID').isin(y))

In [74]:
from pyspark.sql.functions import col, count

# Group the filtered_rating DataFrame by 'Book-Title' and count the 'Book-Rating'
book_rating_counts = filtered_rating.groupBy('Book-Title').agg(count('Book-Rating').alias('book_rating_count'))

# Filter the book_rating_counts DataFrame to keep only rows where 'book_rating_count' is greater than or equal to 50
r = book_rating_counts.filter(col('book_rating_count') >= 50)

# Get the 'Book-Title' values from the filtered DataFrame
famous_books = r.select('Book-Title').rdd.flatMap(lambda x: x).collect()

In [75]:
final_ratings = filtered_rating.filter(col('Book-Title').isin(famous_books))

In [76]:
# Pivot the DataFrame
pt = final_ratings.groupby('Book-Title').pivot('User-ID').agg({'Book-Rating': 'first'})


In [77]:
# Replace null values with 0
pt = pt.fillna(0)

In [78]:
# Convert PySpark DataFrame to RDD
pt_rdd = pt.rdd.zipWithIndex()

# Filter the RDD to get the row at any index 
row= pt_rdd.filter(lambda x: x[1] == 10).map(lambda x: x[0]).collect()

# If there's a row at index, extract the Book-Title
if row:
    book_title_ = row[0]['Book-Title']
    print(book_title_)
else:
    print("No row found at index")


24/05/06 00:38:45 WARN DAGScheduler: Broadcasting large task binary with size 1341.5 KiB


A Fine Balance


In [79]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col

In [80]:
# Rename the columns with integer names to avoid errors
new_column_names = ["col_" + str(i) for i in range(len(pt.columns))]
pt_renamed = pt.toDF(*new_column_names)


In [81]:
from pyspark.sql.functions import col, udf, isnan, when
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler

# Convert columns to double type
pt_renamed = pt.select([col(c).cast("double").alias(c) for c in pt.columns])

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=pt_renamed.columns[1:], outputCol="features")
vector_df = assembler.transform(pt_renamed)

# Calculate norm of each feature vector
norm_udf = udf(lambda v: float(v.norm(2)), DoubleType())

# Apply the UDF to calculate the norm of the features column
vector_df = vector_df.withColumn('features_norm', norm_udf(col('features')))

# Replace NaN values in 'features_norm' column with 0
vector_df = vector_df.withColumn('features_norm', when(isnan(vector_df['features_norm']), 0).otherwise(vector_df['features_norm']))

# Check if 'features_norm' column exists and its data type is compatible
if 'features_norm' in vector_df.columns and vector_df.schema['features_norm'].dataType == DoubleType():
    # Check for zero norm vectors
    zero_norm_count = vector_df.filter(vector_df.features_norm == 0.0).count()
    print("Number of zero norm vectors:", zero_norm_count)

    # Define a user-defined function (UDF) to calculate cosine similarity
    cosine_similarity_udf = udf(lambda v1, v2: float(v1.dot(v2) / (v1.norm(2) * v2.norm(2))), DoubleType())

    # Cross join to calculate cosine similarity between all pairs of vectors
    similarity_score = vector_df.alias("df1").crossJoin(
        vector_df.alias("df2")
    ).select(
        col("df1.Book-Title").alias("Book-Title1"),
        col("df2.Book-Title").alias("Book-Title2"),
        cosine_similarity_udf("df1.features", "df2.features").alias("cosine_similarity")
    )

    # Show the results
    similarity_score.show()

    # Now, let's proceed with the recommendations code

    def recommend(book_name, similarity_score):
        # Find similar items based on similarity score
        similar_items = similarity_score.filter((col("Book-Title1") == book_name) | (col("Book-Title2") == book_name)) \
                                        .filter(col("Book-Title1") != col("Book-Title2")) \
                                        .orderBy(col("cosine_similarity").desc()) \
                                        .limit(4) \
                                        .collect()

        data = []
        for row in similar_items:
            similar_book = row["Book-Title1"] if row["Book-Title2"] == book_name else row["Book-Title2"]
            temp_df = pt.filter(pt['Book-Title'] == similar_book)
            item = [
                similar_book,
                temp_df.select("Book-Author").distinct().first()[0],
                temp_df.select("Image-URL-M").distinct().first()[0]
            ]
            data.append(item)

        return data

    # Call recommend function
    recommendations = recommend('1984', similarity_score)
else:
    print("'features_norm' column doesn't exist or has incompatible data type.")



24/05/06 00:38:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/05/06 00:39:04 WARN DAGScheduler: Broadcasting large task binary with size 1906.0 KiB
24/05/06 00:39:05 WARN TaskSetManager: Lost task 0.0 in stage 304.0 (TID 705) (10.0.8.186 executor 0): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$Lambda$2231/0x00007a2740aab598`: (struct<100459:double,100644:double,100846:double,100906:double,101209:double,101851:double,101876:double,102275:double,102647:double,102702:double,102967:double,104399:double,104429:double,10447:double,104636:double,104665:double,105374:double,105517:double,105979:double,106225:double,106816:double,107021:double,107301:double,107453:double,107784:double,107951:double,108005:double,10819:double,108285:double,108352:double,109461:double,109574:double,109955:double,110029

Py4JJavaError: An error occurred while calling o4913.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 304.0 failed 4 times, most recent failure: Lost task 0.3 in stage 304.0 (TID 708) (10.0.8.186 executor 0): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$2231/0x00007a2740aab598`: (struct<100459:double,100644:double,100846:double,100906:double,101209:double,101851:double,101876:double,102275:double,102647:double,102702:double,102967:double,104399:double,104429:double,10447:double,104636:double,104665:double,105374:double,105517:double,105979:double,106225:double,106816:double,107021:double,107301:double,107453:double,107784:double,107951:double,108005:double,10819:double,108285:double,108352:double,109461:double,109574:double,109955:double,110029:double,110483:double,110912:double,110934:double,110973:double,111174:double,111947:double,112001:double,112026:double,112881:d

In [ ]:
# Convert PySpark DataFrame to Pandas DataFrame
pandas_df = popular_df.toPandas()

# Save Pandas DataFrame as Pickle file
pandas_df.to_pickle("popular.pkl")

# Print the first few rows of the Pandas DataFrame
print(pandas_df.head())

In [ ]:
# Drop duplicates based on the 'Book-Title' column
books_without_duplicates = books.dropDuplicates(['Book-Title'])

# Show the resulting DataFrame
books_without_duplicates.show()

In [ ]:
# Save pt DataFrame
pt.write.parquet("/home/hadoop/pt.parquet")

# Save books DataFrame
books.write.parquet("/home/hadoop/books.parquet")

# Save similarity_score DataFrame
similarity_score.write.parquet("/home/hadoop/similarity_scores.parquet")
